In [117]:
import pandas as pd

from collections import namedtuple
from glob import glob

In [14]:
all_reviews_path = '../../aspects/data/aspects/Reviews-9-products/'

In [15]:
reviews_paths = glob(all_reviews_path + '*')

In [16]:
reviews_paths

['../../aspects/data/aspects/Reviews-9-products/MicroMP3.txt',
 '../../aspects/data/aspects/Reviews-9-products/Canon S100.txt',
 '../../aspects/data/aspects/Reviews-9-products/norton.txt',
 '../../aspects/data/aspects/Reviews-9-products/Diaper Champ.txt',
 '../../aspects/data/aspects/Reviews-9-products/Hitachi router.txt',
 '../../aspects/data/aspects/Reviews-9-products/ipod.txt',
 '../../aspects/data/aspects/Reviews-9-products/Nokia 6600.txt',
 '../../aspects/data/aspects/Reviews-9-products/Linksys Router.txt',
 '../../aspects/data/aspects/Reviews-9-products/Canon PowerShot SD500.txt']

In [149]:
def get_aspects(reviews_path: str) -> pd.DataFrame:
    print(reviews_path)
    all_aspects = []
    with open(reviews_path, 'r') as review_file:
        for line in review_file:
            aspects = [x for x in line.split('##')[0].split(',') if x]
            for aspect in aspects:
                aspect = aspect.strip()
                aspect = aspect.replace('[u]', '').replace('[s]', '').replace('[p]', '')
                aspect = aspect.replace('[cs]', '')
                aspect = aspect.replace('{', '[').replace('}', ']')
                if aspect:
                    all_aspects.append(get_sentiment_from_aspect_sentiment_text(aspect))
    return pd.DataFrame(all_aspects)

In [150]:
def get_sentiment_from_aspect_sentiment_text(aspect_with_sentiment: str):
    aspect_splitted = aspect_with_sentiment.split('[')
    aspect = aspect_splitted[0]
    try:
        sentiment_str = aspect_splitted[1].replace(']', '')
    except IndexError:
        print(f'Error with: {aspect_with_sentiment}')
        raise Exception
    if '-' in sentiment_str:
        sentiment = -1
    elif '+' in sentiment_str:
        if len(sentiment_str) > 1:
            sentiment = int(sentiment_str)
        else:
            sentiment = 1
    else:
        sentiment = int(sentiment_str)
    return AspectSentiment(aspect=aspect.lower(), sentiment=sentiment)

In [160]:
for reviews_path in reviews_paths:
    df = get_aspects(reviews_path)

../../aspects/data/aspects/Reviews-9-products/MicroMP3.txt
../../aspects/data/aspects/Reviews-9-products/Canon S100.txt
../../aspects/data/aspects/Reviews-9-products/norton.txt
../../aspects/data/aspects/Reviews-9-products/Diaper Champ.txt
../../aspects/data/aspects/Reviews-9-products/Hitachi router.txt
../../aspects/data/aspects/Reviews-9-products/ipod.txt
../../aspects/data/aspects/Reviews-9-products/Nokia 6600.txt
../../aspects/data/aspects/Reviews-9-products/Linksys Router.txt
../../aspects/data/aspects/Reviews-9-products/Canon PowerShot SD500.txt
